# Capstone Project-The Battle of Neighbourhoods Final

## Introduction- Business opportunity in Selangor

Selangor also known by its Arabic honorific Darul Ehsan, or "Abode of Sincerity", is one of the 13 states of Malaysia. It is on the west coast of Peninsular Malaysia and is bordered by Perak to the north, Pahang to the east, Negeri Sembilan to the south and the Strait of Malacca to the west. Selangor surrounds the federal territories of Kuala Lumpur and Putrajaya, both of which were previously part of it.

The state capital of Selangor is Shah Alam and its royal capital is Klang. Petaling Jaya and Subang Jaya received city status in 2006 and 2019, respectively.Selangor is one of four Malaysian states that contain more than one city with official city status; the others are Sarawak, Johor, and Penang.

The state of Selangor has the largest economy in Malaysia in terms of gross domestic product (GDP), with RM 239.968 billion (roughly US$55.5 billion) in 2015, comprising 22.6 percent of the country's GDP. It is the most developed state in Malaysia, and has the largest population and the lowest poverty rate in the country.

Kuala Lumpur is the capital of Malaysia. Its modern skyline is dominated by the 451m-tall Petronas Twin Towers, a pair of glass-and-steel-clad skyscrapers with Islamic motifs. The towers also offer a public skybridge and observation deck. The city is also home to British colonial-era landmarks such as the Kuala Lumpur Railway Station and the Sultan Abdul Samad Building

Kuala Lumpur is the cultural, financial and economic centre of Malaysia. It is also home to the Parliament of Malaysia and the official residence of the Yang di-Pertuan Agong, the Istana Negara. It first developed as a town serving the tin mines of the region circa 1857, before it became the capital of Selangor in 1880 until 1978, and the capital of Malaya before the formation of Malaysia. The city remained the seat of the executive and judicial branches of the federal government until these were relocated to Putrajaya in early 1999. However, some sections of the political bodies still remain in Kuala Lumpur.

## Business Problem and Sources:

The Wikipedia page https://en.wikipedia.org/wiki/Selangor#List_of_districts is the major source of data that is being used to obtain all the districts of Selangor and Foursquare API to access the venues in the neighbourhoods. List all the major parts and restaurant in Selangor. Which area are the most restaurant located. Folium visualization library can be used to visualize the clusters superimposed on the map of Selangor. These clusters will be further analyzed to help business owners selecting a potential location to open-up Hotels, Shopping Malls, Restaurants or Coffee shops.

## Target Audience:

This analyst aims to people who want to start run their first business in Selangor. The below dataset will give them an idea on how to select a better place and what type of business is better.

This report also suits people who already own a business and want to make a change for their business. The below data set will provide them some valuable information for them to make decision.

In [1]:
import numpy as np
import re
import json
import requests
from bs4 import BeautifulSoup

import pandas as pd
#display all rows
pd.set_option('display.max_rows', None)
#display all columns
pd.set_option('display.max_columns', None)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

from geopy.geocoders import Nominatim

import folium

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'YXKLDGWS5IBXLEFVYCY1M3EOZCYNHI1JWJ4RZYR1O1CDBRNE' # your Foursquare ID
CLIENT_SECRET = 'IWRW0I5PRT1BKHAHXSMCJ2H2H25TRW02ZBCO1J4MU3025DFS' # your Foursquare Secret
ACCESS_TOKEN = 'EPC2NWX1UGE2IFKRXA33TX23ZLXQTLHSDXFFIFEJ21S03RWL' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YXKLDGWS5IBXLEFVYCY1M3EOZCYNHI1JWJ4RZYR1O1CDBRNE
CLIENT_SECRET:IWRW0I5PRT1BKHAHXSMCJ2H2H25TRW02ZBCO1J4MU3025DFS


In [3]:
url='https://en.wikipedia.org/wiki/List_of_districts_in_Malaysia_by_population'
mdf=pd.read_html(url, header=0)[0]
mdf.head(10)

,Rank,Local government area,State,Total population,Status,Notes,Urban area
0,1,Kuala Lumpur,Federal Territories,1588750,City,Federal capital of Malaysia,Greater Kuala Lumpur (Klang Valley)
1,2,Seberang Perai,Penang,818197,City,"Includes Butterworth, Bukit Mertajam, Batu Kaw...",Greater Penang
2,3,Kajang,Selangor,795522,Municipality,NaN,Greater Kuala Lumpur (Klang Valley)
3,4,Klang,Selangor,744062,Municipality,NaN,Greater Kuala Lumpur (Klang Valley)
4,5,Subang Jaya,Selangor,708296,City,Part of Petaling District,Greater Kuala Lumpur (Klang Valley)
5,6,Penang Island,Penang,708127,City,"Includes George Town, the capital of Penang",Greater Penang
6,7,Ipoh,Perak,657892,City,Capital of Perak,Greater Ipoh (Kinta Valley)
7,8,Petaling Jaya,Selangor,613977,City,Part of Petaling District,Greater Kuala Lumpur (Klang Valley)
8,9,Selayang,Selangor,542409,Municipality,NaN,Greater Kuala Lumpur (Klang Valley)
9,10,Shah Alam,Selangor,541306,City,Capital of Selangor,Greater Kuala Lumpur (Klang Valley)


In [4]:
mf = mdf.drop(['Rank','Notes','Urban area'], axis=1)
mf.head()

,Local government area,State,Total population,Status
0,Kuala Lumpur,Federal Territories,1588750,City
1,Seberang Perai,Penang,818197,City
2,Kajang,Selangor,795522,Municipality
3,Klang,Selangor,744062,Municipality
4,Subang Jaya,Selangor,708296,City


In [5]:
geolocator = Nominatim(user_agent="Selangor_explorer")
mf['Co-ordinates']= mf['Local government area'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
mf[['Latitude', 'Longitude']] = mf['Co-ordinates'].apply(pd.Series)
mf['State'] = 'Selangor'
mf.head()

,Local government area,State,Total population,Status,Co-ordinates,Latitude,Longitude
0,Kuala Lumpur,Selangor,1588750,City,"(3.1516964, 101.6942371)",3.151696,101.694237
1,Seberang Perai,Selangor,818197,City,"(5.35397935, 100.45734643656445)",5.353979,100.457346
2,Kajang,Selangor,795522,Municipality,"(2.98320905, 101.79035124931443)",2.983209,101.790351
3,Klang,Selangor,744062,Municipality,"(49.3198, 6.3722)",49.319800,6.372200
4,Subang Jaya,Selangor,708296,City,"(3.051487, 101.5823339)",3.051487,101.582334


In [6]:
address = 'Selangor'

geolocator = Nominatim(user_agent="Selangor_explorer")
location = geolocator.geocode(address)
Selangor_latitude = location.latitude
Selangor_longitude = location.longitude
print('The geograpical coordinates of Selangor are {}, {}.'.format(Selangor_latitude, Selangor_longitude))

The geograpical coordinates of Selangor are 3.2083304, 101.304146.


In [7]:
!pip install folium
import folium

mf_Brgh = folium.Map(location=[Selangor_latitude, Selangor_longitude], zoom_start=12)


for lat, lng, label in zip(mf['Latitude'], mf['Longitude'], 
                            mf['Local government area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(mf_Brgh)
mf_Brgh

In [8]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Local government area', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

In [9]:
Brgh_Venues = getVenues(names=mf['Local government area'],
                        latitudes=mf['Latitude'],
                        longitudes=mf['Longitude'])

Kuala Lumpur
Seberang Perai
Kajang
Klang
Subang Jaya
Penang Island
Ipoh
Petaling Jaya
Selayang
Shah Alam
Iskandar Puteri
Johor Bahru
Malacca City
Ampang Jaya
Kota Kinabalu
Sungai Petani
Kuantan
Alor Setar
Tawau
Sandakan
Kuala Terengganu
Kuching
Kota Bharu
Seremban
Kulim
Padawan
Taiping
Miri
Kulai
Kangar
Kuala Langat
Kubang Pasu
Bintulu
Manjung
Batu Pahat
Sepang
Kuala Selangor
Muar
Nilai
Lahad Datu
Hulu Selangor
Kinabatangan
Pasir Mas
Penampang
Alor Gajah
Keningau
Kluang
Kemaman
Sibu
Temerloh


In [10]:
Brgh_Venues.groupby('Local government area').count()

,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Local government area,,,,,,
Alor Gajah,99,99,99,99,99,99
Alor Setar,100,100,100,100,100,100
Ampang Jaya,100,100,100,100,100,100
Batu Pahat,9,9,9,9,9,9
Bintulu,49,49,49,49,49,49
Hulu Selangor,6,6,6,6,6,6
Ipoh,100,100,100,100,100,100
Iskandar Puteri,100,100,100,100,100,100
Johor Bahru,100,100,100,100,100,100


In [11]:
Selangor_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid =  Brgh_Venues['Local government area']

Selangor_Brgh_onehot.insert(0, 'Local government area', mid)

Selangor_Brgh_onehot.head()

,Local government area,Accessories Store,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cave,Cemetery,Chettinad Restaurant,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Courthouse,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Shop,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racetrack,Recreation Center,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toll Plaza,Town,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel & Transport,Udon Restaurant

In [12]:
Brgh_grouped = Selangor_Brgh_onehot.groupby('Local government area').mean().reset_index()
Brgh_grouped

,Local government area,Accessories Store,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cave,Cemetery,Chettinad Restaurant,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Courthouse,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Shop,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racetrack,Recreation Center,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toll Plaza,Town,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel & Transport,Udon Restaurant

In [13]:
num_top_venues = 5

for brgh in Brgh_grouped['Local government area']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Local government area'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Alor Gajah________
              venue  freq
0  Malay Restaurant  0.16
1  Asian Restaurant  0.08
2              Café  0.07
3       Pizza Place  0.04
4       Coffee Shop  0.04


_________Alor Setar________
                venue  freq
0    Malay Restaurant  0.09
1    Asian Restaurant  0.08
2  Chinese Restaurant  0.05
3     Thai Restaurant  0.05
4          Food Truck  0.05


_________Ampang Jaya________
              venue  freq
0  Malay Restaurant  0.09
1             Hotel  0.08
2       Coffee Shop  0.05
3  Asian Restaurant  0.04
4              Café  0.04


_________Batu Pahat________
              venue  freq
0  Malay Restaurant  0.44
1        Toll Plaza  0.11
2    Breakfast Spot  0.11
3        Steakhouse  0.11
4            Bistro  0.11


_________Bintulu________
              venue  freq
0              Café  0.12
1             Hotel  0.10
2             Beach  0.10
3       Coffee Shop  0.06
4  Asian Restaurant  0.06


_________Hulu Selangor________
          venue  freq
0    Fo

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Local government area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brgh_venues_sorted = pd.DataFrame(columns=columns)
brgh_venues_sorted['Local government area'] = Brgh_grouped['Local government area']

for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)

brgh_venues_sorted.head(8)

,Local government area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Alor Gajah,Malay Restaurant,Asian Restaurant,Café,Pizza Place,Coffee Shop
1,Alor Setar,Malay Restaurant,Asian Restaurant,Café,Thai Restaurant,Food Truck
2,Ampang Jaya,Malay Restaurant,Hotel,Coffee Shop,Italian Restaurant,Chinese Restaurant
3,Batu Pahat,Malay Restaurant,Steakhouse,Bistro,Asian Restaurant,Toll Plaza
4,Bintulu,Café,Hotel,Beach,Coffee Shop,Asian Restaurant
5,Hulu Selangor,Food Truck,Night Market,Farm,Motel,Beach
6,Ipoh,Coffee Shop,Chinese Restaurant,Café,Dessert Shop,Indian Restaurant
7,Iskandar Puteri,Theme Park Ride / Attraction,Café,Coffee Shop,Theme Park,Indian Restaurant


In [15]:
kclusters = 3

brgh_grouped_clustering = Brgh_grouped.drop('Local government area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brgh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 2, 0, 0, 0, 0, 0, 2])

In [16]:
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(mf,brgh_venues_sorted, on='Local government area')
Borough_merged

,Local government area,State,Total population,Status,Co-ordinates,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Kuala Lumpur,Selangor,1588750,City,"(3.1516964, 101.6942371)",3.151696,101.694237,0,Hotel,Café,Boutique,Hotel Bar,Shopping Mall
1,Seberang Perai,Selangor,818197,City,"(5.35397935, 100.45734643656445)",5.353979,100.457346,0,Chinese Restaurant,Coffee Shop,Malay Restaurant,Food Truck,Asian Restaurant
2,Kajang,Selangor,795522,Municipality,"(2.98320905, 101.79035124931443)",2.983209,101.790351,2,Malay Restaurant,Restaurant,Chinese Restaurant,Asian Restaurant,Indian Restaurant
3,Klang,Selangor,744062,Municipality,"(49.3198, 6.3722)",49.319800,6.372200,0,Bakery,Vacation Rental,French Restaurant,Resort,History Museum
4,Subang Jaya,Selangor,708296,City,"(3.051487, 101.5823339)",3.051487,101.582334,0,Coffee Shop,Japanese Restaurant,Convenience Store,Bakery,Ice Cream Shop
5,Penang Island,Selangor,708127,City,"(5.3696832, 100.25281058745169)",5.369683,100.252811,0,Chinese Restaurant,Asian Restaurant,Café,Food Truck,Seafood Restaurant
6,Ipoh,Selangor,657892,City,"(4.5986817, 101.0900236)",4.598682,101.090024,0,Coffee Shop,Chinese Restaurant,Café,Dessert Shop,Indian Restaurant
7,Petaling Jaya,Selangor,613977,City,"(3.0991752, 101.6450232)",3.099175,101.645023,0,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Café,Malay Restaurant
8,Selayang,Selangor,542409,Municipality,"(3.2537121, 101.6539012)",3.253712,101.653901,0,Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Malay Restaurant,Café
9,Shah Alam,Selangor,541306,City,"(3.0733644, 101.5195149)",3.073364,101.519515,0,Malay Restaurant,Boutique,Restaurant,Food Truck,Asian Restaurant


In [17]:
map_clusters = folium.Map(location=[Selangor_latitude, Selangor_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, pop in zip(Borough_merged['Latitude'],
                                  Borough_merged['Longitude'],
                                  Borough_merged['Local government area'],
                                  Borough_merged['Cluster Label'],
                                  Borough_merged['Total population']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + " " + "Total population " + str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 0, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Local government area,State,Total population,Co-ordinates,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Kuala Lumpur,Selangor,1588750,"(3.1516964, 101.6942371)",101.694237,0,Hotel,Café,Boutique,Hotel Bar,Shopping Mall
1,Seberang Perai,Selangor,818197,"(5.35397935, 100.45734643656445)",100.457346,0,Chinese Restaurant,Coffee Shop,Malay Restaurant,Food Truck,Asian Restaurant
3,Klang,Selangor,744062,"(49.3198, 6.3722)",6.372200,0,Bakery,Vacation Rental,French Restaurant,Resort,History Museum
4,Subang Jaya,Selangor,708296,"(3.051487, 101.5823339)",101.582334,0,Coffee Shop,Japanese Restaurant,Convenience Store,Bakery,Ice Cream Shop
5,Penang Island,Selangor,708127,"(5.3696832, 100.25281058745169)",100.252811,0,Chinese Restaurant,Asian Restaurant,Café,Food Truck,Seafood Restaurant
6,Ipoh,Selangor,657892,"(4.5986817, 101.0900236)",101.090024,0,Coffee Shop,Chinese Restaurant,Café,Dessert Shop,Indian Restaurant
7,Petaling Jaya,Selangor,613977,"(3.0991752, 101.6450232)",101.645023,0,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Café,Malay Restaurant
8,Selayang,Selangor,542409,"(3.2537121, 101.6539012)",101.653901,0,Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Malay Restaurant,Café
9,Shah Alam,Selangor,541306,"(3.0733644, 101.5195149)",101.519515,0,Malay Restaurant,Boutique,Restaurant,Food Truck,Asian Restaurant
10,Iskandar Puteri,Selangor,529074,"(1.4226388, 103.6518138)",103.651814,0,Theme Park Ride / Attraction,Café,Coffee Shop,Theme Park,Indian Restaurant


In [19]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 1, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Local government area,State,Total population,Co-ordinates,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
47,Kemaman,Selangor,166750,"(4.1927895, 103.11948723134745)",103.119487,1,Pizza Place,Zoo,Flower Shop,Fast Food Restaurant,Field


In [20]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Local government area,State,Total population,Co-ordinates,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Kajang,Selangor,795522,"(2.98320905, 101.79035124931443)",101.790351,2,Malay Restaurant,Restaurant,Chinese Restaurant,Asian Restaurant,Indian Restaurant
16,Kuantan,Selangor,427515,"(3.7985637, 103.32199)",103.321990,2,Malay Restaurant,Park,Coffee Shop,Hotel,Asian Restaurant
20,Kuala Terengganu,Selangor,337553,"(5.3297015, 103.138313)",103.138313,2,Malay Restaurant,Asian Restaurant,Café,Breakfast Spot,Scenic Lookout
22,Kota Bharu,Selangor,314964,"(6.1256053, 102.2393455)",102.239345,2,Malay Restaurant,Coffee Shop,Asian Restaurant,Thai Restaurant,Seafood Restaurant
24,Kulim,Selangor,281260,"(5.3663769, 100.5525705)",100.552571,2,Malay Restaurant,Asian Restaurant,Food Truck,Coffee Shop,Chinese Restaurant
29,Kangar,Selangor,225590,"(6.4389312, 100.1944838)",100.194484,2,Malay Restaurant,Asian Restaurant,Café,Thai Restaurant,Convenience Store
30,Kuala Langat,Selangor,220214,"(2.8365619, 101.4961821)",101.496182,2,Chinese Restaurant,Malay Restaurant,Asian Restaurant,Café,Indian Restaurant
31,Kubang Pasu,Selangor,214479,"(6.3356376, 100.3723218)",100.372322,2,Malay Restaurant,Thai Restaurant,Asian Restaurant,Restaurant,Café
33,Manjung,Selangor,211113,"(4.334761, 100.6342662)",100.634266,2,Breakfast Spot,Malay Restaurant,Resort,Fishing Spot,Flower Shop
34,Batu Pahat,Selangor,209461,"(1.933333, 103.0)",103.000000,2,Malay Restaurant,Steakhouse,Bistro,Asian Restaurant,Toll Plaza


## Results
Results of the above analysis and clustering cam be summarized:

1.	The most common venue in Selangor is Malay Restaurant, Asian Restaurant and Cafe.
2.	In the second list we can see the most common results is Pizza Shop, follow by Zoo and Flower Shop.
3.	Refer to above data analyst, we can confirm Selangor residents more prefer to go to restaurant rather than others.

## Discussion

Looking at the data, Alor Gajah and Alor Setar are the best places outside of Kuala Lumpur where a new venue is worth opening. However, a lot of information is not considered, and cannot be obtained from Foursquare Developer.

Higher ethnic presence in a given borough can and will influence the popularity of a given cuisine.
Closer proximity to Inner boroughs and better transport links allows people to travel to the neighbouring borough and impact the measurements.

Many small venues are not registered in Foursquare and are marketed via word-of-mouth, and are not taken into account
Regardless, the analysis provided an insight into what people like and opt for, when it comes to going out in their own neighbourhoods.

## Conclusion

Objective of this project was to analyze the neighbourhoods of Selangor and create a clustering model to suggest potential places to start a new business based on the category. The neighbourhood’s data was obtained from an online source and the Foursquare API was used to find the major venues in each neighbourhood.

A few examples for the applications that the clusters can be used for have also been discussed. A map showing the clusters have been provided. Both these can be used by stakeholders to decide the location for the business. But the data have some of the information was not reliable due to Foursquare API not popular using in Selangor, so we need more data and other parts to take in for consideration before making any decision.